## 參考技術
#### Tracking:
1. https://github.com/LeonLok/Deep-SORT-YOLOv4
2. https://github.com/theAIGuysCode/yolov4-deepsort

# Running the Tracker with YOLOv4

In [ ]:
!python save_model.py --model yolov4

In [ ]:
"""
    執行
    !python object_tracker.py --video ./data/video/{input_video} --output ./outputs/{output_video} --model yolov4 --dont_show True --info_output ./outputs/{outout_csv}
    其中
    input_video 輸入影片名稱, 
    output_video 輸出影片名稱, 
    output_csv 輸出tracking結果csv 
"""
!python object_tracker.py --video ./data/video/cars.mp4 --output ./outputs/cars.avi --model yolov4 --dont_show True --info_output ./outputs/cars.csv

In [ ]:
"""
    依照output_csv所tracking的資料結果，重新讀取相關的video，來輸出最後的影片結果

    前置參數
    input_video = './data/video/ocean_city-fps30.mp4'
    output_video = './outputs/full_25_super_ocean_city-fps30.avi'
    deepsort_result_csv = './outputs/ocean_city-fps30.csv'
    backend_image_path = './outputs/IMG_4587.JPG'
    
    參數說明
    input_video 輸入的影片路徑 (同上個cell的input_video)
    output_video 最後的輸出影片path+fileName
    deepsort_result_csv tracking結果csv (同上個cell的output_csv)
    backend_image_path 準備底圖，底圖需要淨空會動的物件，比如人、車、寵物
"""

from google.colab.patches import cv2_imshow
import cv2
import pandas as pd
from PIL import Image

input_video = './data/video/ocean_city-fps30.mp4'
output_video = './outputs/full_25_super_ocean_city-fps30.avi'
deepsort_result_csv = './outputs/ocean_city-fps30.csv'
backend_image_path = './outputs/IMG_4587.JPG'

track_info = pd.read_csv(deepsort_result_csv, index_col='Unnamed: 0')

try:
    vid = cv2.VideoCapture(int(input_video))
except:
    vid = cv2.VideoCapture(input_video)

out = None
input_max = 25

obj_data = track_info[['appear_time','frame','tracker_id','bbox_xmin','bbox_ymin','bbox_xmax','bbox_ymax']]
tracker_id_list = obj_data['tracker_id'].unique()

groups = []
for group_index in range(input_max):
  group_ids = tracker_id_list[group_index::input_max]
  group_obj = obj_data[obj_data['tracker_id'].isin(group_ids)].sort_values(['tracker_id','frame'])
  groups.append(group_obj)

groups_frames = []
for group_info in groups:
    groups_frames.append(group_info.shape[0])
    
max_frames = max(groups_frames)

# by default VideoCapture returns float instead of int
width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
output_fps = int(vid.get(cv2.CAP_PROP_FPS))
codec = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_video, codec, output_fps, (width, height))

backend_index = 0
for backend_index in range(max_frames):
  backend = backend_image = cv2.imread(backend_image_path)

  for group_obj in groups:
    if group_obj.shape[0] < backend_index:
      continue
    
    obj = group_obj.iloc[backend_index]

    frame_index = obj['frame'] - 1
    bbox_xmin = obj['bbox_xmin']
    bbox_ymin = obj['bbox_ymin']
    bbox_xmax = obj['bbox_xmax']
    bbox_ymax = obj['bbox_ymax']

    vid.set(cv2.CAP_PROP_POS_FRAMES,frame_index)
    ret, frame = vid.read()

    if backend is None:
      backend = frame.copy()
      backend.fill(200)

    backend[bbox_ymin:bbox_ymax, bbox_xmin:bbox_xmax] = frame[bbox_ymin:bbox_ymax, bbox_xmin:bbox_xmax]
    cv2.putText(backend, 'id:'+str(obj.tracker_id) + " " + str(obj.appear_time),(bbox_xmin, int(bbox_ymin-10)),0, 0.75, (0,0,0),2)
  out.write(backend)
cv2.destroyAllWindows()



